In [2]:
import cv2
import numpy as np
import math
import random

# ==========================================================
# KONFIGURASI GAME GLOBAL
# ==========================================================
PADDLE_HEIGHT = 110
PADDLE_WIDTH = 18
BALL_RADIUS = 10

BASE_PADDLE_SPEED = 10
BASE_BALL_SPEED_X = 7
BASE_BALL_SPEED_Y = 5

PLAYER_LIVES_START = 3
ENEMY_LIVES_START = 3

POWERUP_DURATION_FRAMES = 300      # durasi efek power-up
POWERUP_COOLDOWN_FRAMES = 400      # jeda sebelum zona aktif kembali

# ==========================================================
# WARNA 
# ==========================================================
PINK_SOFT   = (255, 182, 193)
PINK_MED    = (255, 105, 180)
PINK_DARK   = (180,  80, 120)
BLUE_SOFT   = (255, 204, 153)
BLUE_MED    = (255, 153,  51)
WHITE_SOFT  = (255, 250, 250)
PURPLE_PAST = (230, 200, 255)
YELLOW_SOFT = (200, 230, 255)

# ==========================================================
# DETEKSI GESTURE TANGAN
# ==========================================================
def detect_hand_gesture(cam_frame, display_frame):
    """
    cam_frame : frame BGR dari kamera untuk proses deteksi
    display_frame : frame tampilan yang akan diberi overlay
    return : (gesture, display_frame)
    Gesture berupa 'open', 'fist', atau None
    """
    h, w, _ = cam_frame.shape

    # Menentukan region of interest (ROI) di sisi kanan
    roi_x1 = int(w * 2 / 3)
    roi_y1 = int(h * 1 / 6)
    roi_x2 = w - 10
    roi_y2 = int(h * 5 / 6)

    roi = cam_frame[roi_y1:roi_y2, roi_x1:roi_x2]

    # Menggambar kotak ROI
    cv2.rectangle(display_frame, (roi_x1, roi_y1), (roi_x2, roi_y2), PINK_MED, 2)
    cv2.putText(display_frame, "Hand Area", (roi_x1, roi_y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, PINK_SOFT, 2)

    if roi.size == 0:
        return None, display_frame

    # Mengubah citra ke HSV untuk segmentasi kulit
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Mengurangi noise dengan Gaussian Blur dan morphological closing
    mask = cv2.GaussianBlur(mask, (5, 5), 0)
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=3)

    # Menampilkan mask kecil di pojok kiri atas (debug)
    small_mask = cv2.resize(mask, (150, 150))
    display_frame[10:160, 10:160] = cv2.cvtColor(small_mask, cv2.COLOR_GRAY2BGR)
    cv2.rectangle(display_frame, (10, 10), (160, 160), PURPLE_PAST, 2)
    cv2.putText(display_frame, "Mask", (10, 180),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, WHITE_SOFT, 1)

    # Mencari kontur terbesar (diasumsikan tangan)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return None, display_frame

    cnt = max(contours, key=cv2.contourArea)
    if cv2.contourArea(cnt) < 2000:
        return None, display_frame

    # Menggambar kontur dan convex hull
    cnt_shifted = cnt + np.array([[roi_x1, roi_y1]])
    cv2.drawContours(display_frame, [cnt_shifted], -1, PURPLE_PAST, 2)

    hull = cv2.convexHull(cnt)
    hull_shifted = hull + np.array([[roi_x1, roi_y1]])
    cv2.drawContours(display_frame, [hull_shifted], -1, PINK_SOFT, 2)

    # Mengambil convexity defects untuk menghitung jumlah jari
    hull_indices = cv2.convexHull(cnt, returnPoints=False)
    if hull_indices is None or len(hull_indices) < 3:
        return None, display_frame

    defects = cv2.convexityDefects(cnt, hull_indices)
    if defects is None:
        return "fist", display_frame

    finger_defects = 0

    # Menghitung jumlah cekungan sebagai indikasi jumlah jari
    for i in range(defects.shape[0]):
        s, e, f, d = defects[i, 0]
        start = tuple(cnt[s][0])
        end   = tuple(cnt[e][0])
        far   = tuple(cnt[f][0])

        # Menghindari pembagian dengan nol
        a = math.dist(start, end)
        b = math.dist(start, far)
        c = math.dist(end, far)
        if b * c == 0:
            continue

        # Menghitung sudut pada titik lekukan
        angle = math.degrees(math.acos((b*b + c*c - a*a) / (2 * b * c)))
        far_global = (far[0] + roi_x1, far[1] + roi_y1)

        # Defect valid jika sudut < 90 derajat dan kedalaman cukup besar
        if angle <= 90 and d > 10000:
            finger_defects += 1
            cv2.circle(display_frame, far_global, 6, (255, 200, 230), -1)

    # Menentukan gesture (open atau fist)
    gesture = "open" if finger_defects >= 2 else "fist"

    cv2.putText(display_frame, f"Defects: {finger_defects}",
                (roi_x1, roi_y2 + 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, PINK_MED, 2)

    return gesture, display_frame

# ==========================================================
# KELAS OBJEK BOLA
# ==========================================================
class Ball:
    def __init__(self, x, y, vx, vy):
        self.x  = x
        self.y  = y
        self.vx = vx
        self.vy = vy

def create_initial_balls(width, height):
    """Membuat bola awal di tengah layar"""
    return [Ball(
        width // 2,
        height // 2,
        BASE_BALL_SPEED_X * random.choice([-1, 1]),
        BASE_BALL_SPEED_Y * random.choice([-1, 1])
    )]

def reset_single_ball(ball, width, height, direction=1):
    """Mereset posisi bola ke tengah"""
    ball.x  = width // 2
    ball.y  = height // 2
    ball.vx = BASE_BALL_SPEED_X * direction
    ball.vy = BASE_BALL_SPEED_Y * random.choice([-1, 1])

# ==========================================================
# FUNGSI POWER-UP
# ==========================================================
def create_powerup_zones(width, height):
    """Membuat dua zona power-up: untuk player dan musuh"""
    zone_w = 80
    zone_h = 80
    margin_y = int(height * 0.25)

    player_zone = {
        "x1": int(width * 0.25) - zone_w // 2,
        "y1": margin_y,
        "x2": int(width * 0.25) + zone_w // 2,
        "y2": margin_y + zone_h,
        "type": random.choice(["speed", "multi"]),
        "owner": "player",
        "cooldown": 0
    }

    enemy_zone = {
        "x1": int(width * 0.75) - zone_w // 2,
        "y1": height - margin_y - zone_h,
        "x2": int(width * 0.75) + zone_w // 2,
        "y2": height - margin_y,
        "type": random.choice(["speed", "multi"]),
        "owner": "enemy",
        "cooldown": 0
    }

    return [player_zone, enemy_zone]

def draw_powerup_zones(frame, zones, theme="pink"):
    """Menggambar zona power-up pada frame"""
    for z in zones:
        if z["cooldown"] > 0:
            color = (120, 120, 120)
        else:
            if z["type"] == "speed":
                color = (130, 220, 255) if theme == "blue" else YELLOW_SOFT
            else:
                color = PINK_MED if theme == "pink" else BLUE_MED

        x1, y1, x2, y2 = z["x1"], z["y1"], z["x2"], z["y2"]
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"{z['owner'][0].upper()}-{z['type']}",
                    (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

def check_powerup_collision(balls, zones):
    """Memeriksa apakah bola memasuki zona power-up"""
    triggered = []
    for z in zones:
        if z["cooldown"] > 0:
            z["cooldown"] -= 1
            continue

        for ball in balls:
            if z["x1"] <= ball.x <= z["x2"] and z["y1"] <= ball.y <= z["y2"]:
                triggered.append((z["owner"], z["type"]))
                z["cooldown"] = POWERUP_COOLDOWN_FRAMES
                break
    return triggered

# ==========================================================
# FUNGSI RESET GAME STATE
# ==========================================================
def reset_game_state(width, height):
    """Mengembalikan seluruh keadaan game ke awal"""
    state = {
        "left_paddle_y": height // 2 - PADDLE_HEIGHT // 2,
        "right_paddle_y": height // 2 - PADDLE_HEIGHT // 2,
        "balls": create_initial_balls(width, height),
        "player_lives": PLAYER_LIVES_START,
        "enemy_lives": ENEMY_LIVES_START,
        "player_speed_mult": 1.0,
        "enemy_speed_mult": 1.0,
        "player_speed_timer": 0,
        "enemy_speed_timer": 0,
        "zones": create_powerup_zones(width, height),
        "game_over": False,
        "winner": None
    }
    return state

# ==========================================================
# PEMBUATAN BACKGROUND ARENA
# ==========================================================
def create_background(frame_shape, mode, theme):
    """Membuat latar belakang untuk mode arena"""
    h, w, _ = frame_shape
    if mode == "camera":
        return None

    if theme == "pink":
        bg = np.full((h, w, 3), (40, 30, 70), dtype=np.uint8)
        court_color = (80, 40, 120)
        line_color = (230, 200, 255)
    else:
        bg = np.full((h, w, 3), (20, 40, 60), dtype=np.uint8)
        court_color = (60, 90, 140)
        line_color = (230, 240, 255)

    margin = 40

    cv2.rectangle(bg, (margin, margin), (w - margin, h - margin), court_color, -1)
    cv2.rectangle(bg, (margin, margin), (w - margin, h - margin), line_color, 2)

    for y in range(margin, h - margin, 25):
        cv2.line(bg, (w // 2, y), (w // 2, y + 15), line_color, 2)

    return bg

# ==========================================================
# MENGGAMBAR HEART (NYAWA)
# ==========================================================
def draw_hearts(frame, lives, x, y):
    """Menggambar indikator nyawa berbentuk hati"""
    total = 3
    heart_color_full = (0, 0, 255)
    heart_color_empty = (180, 180, 180)

    for i in range(total):
        cx = x + i * 40
        cy = y
        color = heart_color_full if i < lives else heart_color_empty
        size = 12

        pts = np.array([
            [cx, cy + size],
            [cx - size, cy],
            [cx - size//2, cy - size],
            [cx, cy - size//3],
            [cx + size//2, cy - size],
            [cx + size, cy],
        ], np.int32)

        pts = pts.reshape((-1, 1, 2))
        cv2.fillPoly(frame, [pts], color)
        cv2.polylines(frame, [pts], True, (255, 255, 255), 2)

# ==========================================================
# START MENU (MENU AWAL)
# ==========================================================
def draw_start_menu(frame):
    """Menggambar tampilan menu awal"""
    h, w, _ = frame.shape

    overlay = frame.copy()
    cv2.rectangle(overlay, (0, 0), (w, h), (50, 20, 70), -1)

    alpha = 0.8
    frame[:] = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

    lines = [
        "==========================",
        "    PINKY PONG EXTREME",
        "==========================",
        "Press 1 : Camera Mode",
        "Press 2 : Arena Mode",
        "Press T : Change Theme",
        "Press Q : Quit",
    ]

    y0 = int(h * 0.25)
    for i, line in enumerate(lines):
        cv2.putText(frame, line,
                    (int(w * 0.12), y0 + i * 40),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1.0, (230, 200, 255), 3)

    return frame

# ==========================================================
# FUNGSI UTAMA (MAIN LOOP)
# ==========================================================
cap = cv2.VideoCapture(1)
ret, frame = cap.read()

if not ret:
    print("Camera error!")
    cap.release()
    raise SystemExit

frame = cv2.flip(frame, 1)
height, width, _ = frame.shape

current_mode = "camera"
current_theme = "pink"
state = reset_game_state(width, height)
last_gesture = None
show_menu = True

print("=== PINKY PONG EXTREME ===")
print("[1] Camera Mode (AR)")
print("[2] Arena Mode (Lapangan)")
print("[T] Toggle Theme (Pink/Blue)")
print("[R] Reset Game")
print("[Q] Quit")
print("Gesture OPEN = paddle naik, FIST = paddle turun")

while True:
    ret, cam_frame = cap.read()
    if not ret:
        break

    cam_frame = cv2.flip(cam_frame, 1)
    base_frame = cam_frame.copy()

    # ======================================================
    # MENU AWAL
    # ======================================================
    if show_menu:
        menu_frame = draw_start_menu(base_frame)
        cv2.imshow("PINKY PONG EXTREME", menu_frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('q'):
            break
        elif key == ord('1'):
            current_mode = "camera"
            show_menu = False
        elif key == ord('2'):
            current_mode = "arena"
            show_menu = False
        elif key == ord('t'):
            current_theme = "blue" if current_theme == "pink" else "pink"
        elif key == ord('r'):
            state = reset_game_state(width, height)
            last_gesture = None

        continue

    # ======================================================
    # MODE GAME SETELAH MENU
    # ======================================================
    if current_mode == "camera":
        display_frame = cam_frame.copy()
    else:
        bg = create_background(cam_frame.shape, current_mode, current_theme)
        display_frame = bg.copy()

    gesture, display_frame = detect_hand_gesture(cam_frame, display_frame)
    if gesture is not None:
        last_gesture = gesture

    # ======================================================
    # TAMPILAN GAME OVER
    # ======================================================
    if state["game_over"]:
        cv2.putText(display_frame, "GAME OVER",
                    (width // 2 - 150, height // 2 - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, PINK_SOFT, 3)

        msg = "YOU WIN" if state["winner"] == "player" else "YOU LOSE"
        cv2.putText(display_frame, msg,
                    (width // 2 - 150, height // 2 + 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.1, WHITE_SOFT, 2)

    # ======================================================
    # LOGIKA GAME (JIKA BELUM GAME OVER)
    # ======================================================
    else:
        # Kecepatan paddle berdasarkan power-up
        player_speed = BASE_PADDLE_SPEED * state["player_speed_mult"]
        enemy_speed  = BASE_PADDLE_SPEED * state["enemy_speed_mult"]

        # Timer power-up
        if state["player_speed_timer"] > 0:
            state["player_speed_timer"] -= 1
            if state["player_speed_timer"] == 0:
                state["player_speed_mult"] = 1.0

        if state["enemy_speed_timer"] > 0:
            state["enemy_speed_timer"] -= 1
            if state["enemy_speed_timer"] == 0:
                state["enemy_speed_mult"] = 1.0

        # Kontrol paddle pemain
        if last_gesture == "open":
            state["left_paddle_y"] -= int(player_speed)
        elif last_gesture == "fist":
            state["left_paddle_y"] += int(player_speed)

        state["left_paddle_y"] = max(0, min(height - PADDLE_HEIGHT, state["left_paddle_y"]))

        # Paddle musuh mengikuti bola pertama
        if len(state["balls"]) > 0:
            target_y = state["balls"][0].y
            center_y = state["right_paddle_y"] + PADDLE_HEIGHT // 2

            if target_y < center_y:
                state["right_paddle_y"] -= int(enemy_speed)
            elif target_y > center_y:
                state["right_paddle_y"] += int(enemy_speed)

        state["right_paddle_y"] = max(0, min(height - PADDLE_HEIGHT, state["right_paddle_y"]))

        # Update bola
        for ball in state["balls"]:
            ball.x += ball.vx
            ball.y += ball.vy

            # Pantulan atas/bawah
            if ball.y - BALL_RADIUS <= 0 or ball.y + BALL_RADIUS >= height:
                ball.vy *= -1

            # Pantulan paddle kiri
            if (ball.x - BALL_RADIUS <= PADDLE_WIDTH and
                state["left_paddle_y"] <= ball.y <= state["left_paddle_y"] + PADDLE_HEIGHT):

                ball.vx *= -1
                ball.x = PADDLE_WIDTH + BALL_RADIUS + 1

            # Pantulan paddle kanan
            if (ball.x + BALL_RADIUS >= width - PADDLE_WIDTH and
                state["right_paddle_y"] <= ball.y <= state["right_paddle_y"] + PADDLE_HEIGHT):

                ball.vx *= -1
                ball.x = width - PADDLE_WIDTH - BALL_RADIUS - 1

        # Kehilangan nyawa bila bola keluar layar
        for ball in state["balls"][:]:
            if ball.x < 0:
                state["player_lives"] -= 1
                reset_single_ball(ball, width, height, direction=1)

            elif ball.x > width:
                state["enemy_lives"] -= 1
                reset_single_ball(ball, width, height, direction=-1)

        # Pemeriksaan game over
        if state["player_lives"] <= 0:
            state["game_over"] = True
            state["winner"] = "enemy"

        elif state["enemy_lives"] <= 0:
            state["game_over"] = True
            state["winner"] = "player"

        # Power-up
        triggers = check_powerup_collision(state["balls"], state["zones"])

        for owner, ptype in triggers:
            if ptype == "speed":
                if owner == "player":
                    state["player_speed_mult"] = 1.8
                    state["player_speed_timer"] = POWERUP_DURATION_FRAMES
                else:
                    state["enemy_speed_mult"] = 1.8
                    state["enemy_speed_timer"] = POWERUP_DURATION_FRAMES

            elif ptype == "multi":
                if len(state["balls"]) < 3:
                    for _ in range(2):
                        new_ball = Ball(
                            width // 2,
                            random.randint(50, height - 50),
                            BASE_BALL_SPEED_X * random.choice([-1, 1]),
                            BASE_BALL_SPEED_Y * random.choice([-1, 1])
                        )
                        state["balls"].append(new_ball)

    # ======================================================
    # RENDERING SEMUA ELEMEN GAME
    # ======================================================
    # Paddle pemain
    cv2.rectangle(display_frame,
                  (0, state["left_paddle_y"]),
                  (PADDLE_WIDTH, state["left_paddle_y"] + PADDLE_HEIGHT),
                  PINK_SOFT if current_theme == "pink" else BLUE_MED,
                  -1)

    # Paddle musuh
    cv2.rectangle(display_frame,
                  (width - PADDLE_WIDTH, state["right_paddle_y"]),
                  (width, state["right_paddle_y"] + PADDLE_HEIGHT),
                  PINK_DARK if current_theme == "pink" else (200, 160, 100),
                  -1)

    # Bola
    for ball in state["balls"]:
        cv2.circle(display_frame, (int(ball.x), int(ball.y)),
                   BALL_RADIUS + 3,
                   PINK_MED if current_theme == "pink" else BLUE_MED, -1)
        cv2.circle(display_frame, (int(ball.x), int(ball.y)),
                   BALL_RADIUS,
                   WHITE_SOFT, -1)

    # Zona power-up
    draw_powerup_zones(display_frame, state["zones"], theme=current_theme)

    # Nyawa
    draw_hearts(display_frame, state["player_lives"], 20, 40)
    draw_hearts(display_frame, state["enemy_lives"], width - 140, 40)

    # Informasi mode dan tema
    cv2.putText(display_frame,
                f"Mode: {current_mode.upper()} | Theme: {current_theme.upper()}",
                (20, height - 50),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, WHITE_SOFT, 2)

    # Gesture terakhir
    if last_gesture is not None:
        cv2.putText(display_frame, f"Gesture: {last_gesture.upper()}",
                    (20, height - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, PINK_SOFT, 2)

    # Mini-menu instruksi
    cv2.putText(display_frame,
                "[1]Cam [2]Arena [T]Theme [R]Reset [Q]Quit",
                (20, height - 80),
                cv2.FONT_HERSHEY_SIMPLEX, 0.55, WHITE_SOFT, 2)

    cv2.imshow("PINKY PONG EXTREME", display_frame)

    # ======================================================
    # INPUT KEYBOARD
    # ======================================================
    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    elif key == ord('1'):
        current_mode = "camera"
    elif key == ord('2'):
        current_mode = "arena"
    elif key == ord('t'):
        current_theme = "blue" if current_theme == "pink" else "pink"
    elif key == ord('r'):
        state = reset_game_state(width, height)
        last_gesture = None
        show_menu = True

# ==========================================================
# MENUTUP APLIKASI
# ==========================================================
cap.release()
cv2.destroyAllWindows()


=== PINKY PONG EXTREME ===
[1] Camera Mode (AR)
[2] Arena Mode (Lapangan)
[T] Toggle Theme (Pink/Blue)
[R] Reset Game
[Q] Quit
Gesture OPEN = paddle naik, FIST = paddle turun
